# lab 3 - Naive Bayes Classifier

In [514]:
from sklearn.metrics import accuracy_score, f1_score, precision_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, KernelPCA

# Zadanie 1 i Zadanie 2

Zaimplementuj Naiwny Klasyfikator Bayesa dla danych ciągłych zakładając normalny
rozkład prawdopodobieństwa dla każdej z cech z osobna. W implementacji
możesz użyć standardowego interfejsu scikit-learn dla modeli predykcyjnych -
metody fit() oraz predict().

Wyznacz średnią i odchylenie standardowe ciągłej cechy xi dla danej klasy Ck, a następnie oblicz prawdopodobieństwa posterior korzystając z Twierdzenia Bayesa oraz wiarygodności danej wzorem.

In [443]:
from sklearn import datasets
import pandas as pd
import numpy as np

iris = datasets.load_iris()
iris_pd = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])


In [444]:
from math import pi, sqrt, exp, log10

In [451]:
y_iris = iris_pd['target']
X_iris = iris_pd.drop(columns=['target'])

In [498]:
class NaiveBayesGaussian:
    def __init__(self):
        self.data = None
        self.means = {}
        self.vars = {}
        self.classes = []
        self.priors = []
    
    def fit(self, X, y):
        self.data = X.assign(target=y)
        self.classes = np.unique(y)
        n_samples = len(y)
        
        # calculate priors
        for cls in self.classes:
            X_cls = X[y == cls]
            self.means[cls] = np.mean(X_cls, axis=0)
            self.vars[cls] = np.var(X_cls, axis=0)
            self.priors.append(log10(len(X_cls) / n_samples))
        
    
    def calculate_likelihood(self, example):
        likelihood = []
        cols = len(self.data.columns) - 2
        
        for c in self.classes:
            p = 0
            for i in range (0, cols):
            
                var_val = self.vars[c].iat[i]
                mean_val = self.means[c].iat[i]
                
                square = 1/sqrt(2*pi*var_val)
                exponent = exp((-(example[i]-mean_val)**2)/(2*var_val))
                    
                if exponent <= 0:
                    exponent = 1e-9
            
                p += log10(square * exponent)
            likelihood.append(p)
        return likelihood
    
    def predict(self, X_test):
        X_test = X_test.to_numpy()
        predicted = []
        for example in X_test:
            likelihood = self.calculate_likelihood(example)
            posterior = [a + b for a, b in zip(likelihood, self.priors)]
            prediction = [1 if p==max(posterior) else 0 for p in posterior]
            predicted.append(prediction.index(1))
        return self.classes[predicted]


# Zadanie 3
Przetestuj działanie własnej implementacji klasyfikatora dla zbioru danych Iris (4
cechy). Zastosuj losowy podział zbioru danych na część trenignową i testową według
proporcji 0.6, 0.4. Powtórz eksperyment 20-krotnie i zmierz średnie wartości:
- accuracy
- f1-score
- precision

Porównaj wyniki uzyskane z zastosowanie własnej implementacji oraz wyniki uzyskane
za pomocą scikit-learn GaussianNB.


In [583]:
def count_20(model, X, y, test_pct=0.4, text=None):
    results = np.zeros((20, 3))

    for i in range(0, 20):
        X_train, X_test, y_train, y_test = train_test_split(X_iris, y_iris, test_size=test_pct)
        model.fit(X_train, y_train)
        predicted = model.predict(X_test)
        results[i, 0] = accuracy_score(predicted, y_test)
        results[i, 1] = precision_score(predicted, y_test, average='macro')
        results[i, 2] = f1_score(predicted, y_test, average='macro')
    
    mean_res = np.mean(results, axis=0) 
    
    print(text)    
    print(f'accuracy: {mean_res[0]}%')
    print(f'precision: {mean_res[1]}')
    print(f'f1_score: {mean_res[2]}')

In [500]:
bayes = NaiveBayesGaussian()
count_20(bayes, X_iris, y_iris, 'Wyniki zaimplementowanego Naive Bayes: ')

Wyniki zaimplementowanego Naive Bayes: 
accuracy: 0.8916666666666666%
precision: 0.8939150826234386
f1_score: 0.8927736629189118


### Porównanie z biblioteką

In [501]:
clf = GaussianNB()
count_20(clf, X_iris, y_iris, 'Wyniki modelu z biblioteki: ')

Wyniki modelu z biblioteki: 
accuracy: 0.9508333333333331%
precision: 0.9500780514734615
f1_score: 0.949944823615029


Metoda Bayesa z bibliteki osiąga porównywalne wyniki co stworzona przeze mnie.

# Zadanie 4

Przetestuj działanie klasyfikatora dla zbioru danych Breast Cancer (30 cech). Zastosuj trzy podane wyżej miary jakości klasyfikacji. Jako zbiór testowy wykorzystaj 0.3 dostępnego zbioru danych. Zbadaj wpływ:

- skalowania (StandardScaler)
- redukcji wymiaru (PCA i Kernel PCA, dobierz najlepszą wartość n components oraz funkcję jądra na Kernel PCA)
- transformacji Box-Cox (dobierz najlepszą wartość parametru λ)

na średnie wyniki klasyfikacji. Czy tego typu wstępne przetwarzanie zbioru danych wpływa na ich poprawę? Porównaj uzyskaną dokładność klasyfikacji (Naiwny Klasyfikator Bayesa) z dokładnością klasyfikacji uzykaną dla lasów losowych (domyślne wartości hiperparametrów) w tej samej konfiguracji eksperymentu. Omów uzyskane wyniki.

In [461]:
data = load_breast_cancer()
cancer = pd.DataFrame(np.c_[data['data'], data['target']], 
                      columns = np.append(data['feature_names'], ['target']))
y_cancer = cancer['target']
X_cancer = cancer.drop(columns=['target'])

In [450]:
cancer.head(3)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0.0
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0.0
2,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0.0


In [582]:
bayes = NaiveBayesGaussian()
count_20(bayes, X_cancer, y_cancer, 0.3, 'Wyniki zaimplementowanego modelu - dane raka piersi - bez modyfikacji: ')

Wyniki zaimplementowanego modelu - dane raka piersi - bez modyfikacji: 
accuracy: 0.8858333333333333%
precision: 0.8848573019766427
f1_score: 0.8814012226169259


### Skalowanie

In [487]:
scaler = StandardScaler()
scaler.fit(X_cancer)
scaled_X = scaler.transform(X_cancer)

In [587]:
count_20(bayes, scaled_X, y_cancer, 0.3, 'Wyniki zaimplementowanego modelu - dane raka piersi - skalowanie danych ')

Wyniki zaimplementowanego modelu - dane raka piersi - skalowanie danych 
accuracy: 0.8949999999999999%
precision: 0.8963935667738332
f1_score: 0.8946378916238951


### PCA

In [512]:
pca = PCA(n_components=1)
X_cancer_pca = pca.fit(X_cancer)
print(pca.explained_variance_ratio_)

[0.98204467]


Odpowiednie n_components wynosi 1.

In [586]:
count_20(bayes, X_cancer_pca, y_cancer, 0.3,'Wyniki zaimplementowanego modelu - dane raka piersi - pca ')

Wyniki zaimplementowanego modelu - dane raka piersi - pca 
accuracy: 0.8783333333333335%
precision: 0.8786002772530794
f1_score: 0.8749574342370465


### Kernel PCA

In [519]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

kernels = ['linear', 'poly', 'rbf', 'sigmoid']
scores = {}

for kernel in kernels:
    kpca = KernelPCA(kernel=kernel, n_components=1)
    X_kpca = kpca.fit_transform(X_cancer)
    
    svm = SVC()
    score = cross_val_score(svm, X_kpca, y_cancer, cv=5)
    scores[kernel] = score.mean()

print(scores)

{'linear': 0.9068933395435492, 'poly': 0.8858096568855768, 'rbf': 0.6274181027790716, 'sigmoid': 0.6274181027790716}


Najlepsze wyniki dla kernala liniowego.

In [520]:
kpca = KernelPCA(kernel='linear', n_components=1)
X_cancer_kpca = kpca.fit_transform(X_cancer)

In [585]:
count_20(bayes, X_cancer_pca, y_cancer, 0.3, 'Wyniki zaimplementowanego modelu - dane raka piersi - kpca ')

Wyniki zaimplementowanego modelu - dane raka piersi - kpca 
accuracy: 0.8783333333333333%
precision: 0.8778606716130272
f1_score: 0.8749890717952171


### Box-Cox

In [584]:
from sklearn.preprocessing import PowerTransformer

X_cancer_train, X_cancer_test, y_train, y_cancer_test = train_test_split(X_cancer, y_cancer, test_size=0.3)

box_cox_transformer = PowerTransformer(method='box-cox', standardize=True)
X_boxcox = box_cox_transformer.fit_transform(X_cancer + 0.01)

count_20(bayes, X_boxcox, y_cancer_test, 0.3, 'Wyniki zaimplementowanego modelu - dane raka piersi - box-cox ')

Wyniki zaimplementowanego modelu - dane raka piersi - box-cox 
accuracy: 0.8733333333333333%
precision: 0.8746222490183208
f1_score: 0.874142023099872


Różnice jakości modeli z różnymi transformacjami danych są nieznaczne.

# Zadanie 5 - Naiwny Klasyfikator Bayesa z rozkładem Bernoulliego

In [613]:
from sklearn.datasets import fetch_20newsgroups

groups = fetch_20newsgroups()

In [614]:
groups = pd.DataFrame(np.c_[groups['data'], groups['target']],
                      columns = np.append(['text'], ['target']))
y_group = groups['target']
X_group = groups['text']

In [615]:
from sklearn.feature_extraction.text import CountVectorizer

X_train, X_test, y_train, y_test = train_test_split(X_group, y_group, test_size=0.3, random_state=42)

vectorizer = CountVectorizer(binary=True)
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

In [683]:
class BernoulliNaiveBayes:
    def __init__(self):
        self.class_priors = {}
        self.feature_probs = {}
        self.classes = None
        self.alpha = 1 

    def fit(self, X, y):
        self.classes = np.unique(y)
        n_samples, n_features = X.shape
        
        for cls in self.classes:
            X_cls = X[y == cls]
            self.class_priors[cls] = (X_cls.shape[0] + self.alpha) / (n_samples + len(self.classes) * self.alpha)

            
            feature_count = X_cls.sum(axis=0) + self.alpha 
            total_count = X_cls.shape[0] + 2 * self.alpha  
            self.feature_probs[cls] = feature_count / total_count

    def predict(self, X):
        predictions = []
        
        for x in X:
            probs = {}
            for cls in self.classes:
                prior = np.log(self.class_priors[cls]) 
                feature_probs = np.clip(self.feature_probs[cls], 1e-10, 1 - 1e-10)
                x = np.asarray(x)
                likelihood = np.sum(x * np.log(feature_probs)) + np.sum((1 - x) * np.log(1 - feature_probs))
                probs[cls] = prior + likelihood
            
            predictions.append(max(probs, key=probs.get))
        
        return np.array(predictions)

In [684]:
model = BernoulliNaiveBayes()
model.fit(X_train_bow.toarray(), y_train)
predictions = model.predict(X_test_bow.toarray())

In [686]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions)
print(f'Dokładność: {accuracy:.2f}')

Dokładność: 0.65
